<a href="https://colab.research.google.com/github/Shreyas0744/Deep-Reinforcement-Learning-for-Autonomous-Vehicle-Control/blob/main/DRL_for_Autonomous_Vehicle_Control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies for Virtual Display and RL
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gymnasium[box2d] stable-baselines3[extra] pyvirtualdisplay swig > /dev/null 2>&1

# Setup Virtual Display (Crucial for Colab to 'see' the game)
from pyvirtualdisplay import Display
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv
import base64
from pathlib import Path
from IPython.display import HTML
from IPython import display as ipythondisplay

display = Display(visible=0, size=(1400, 900))
display.start()

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [2]:
# Create the Autonomous Driving Environment
def make_env():
    # We use discrete actions for faster initial learning (Steer Left, Right, Gas, Brake, Nothing)
    env = gym.make("CarRacing-v3", render_mode="rgb_array", continuous=False)
    return env

env = DummyVecEnv([make_env])

# Initialize the PPO Agent (The "Brain")
# 'CnnPolicy' is used because the input is a 96x96 image (pixels)
model = PPO("CnnPolicy", env, verbose=1, device="cuda")

# Training Phase
# For an M.Tech project, you'd usually train for 500,000+ steps.
# We'll do 10,000 here so you can see it work quickly.
print("Training the Autonomous Agent...")
model.learn(total_timesteps=10000)
model.save("ppo_car_racing")

<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/lib/python3.12/dist-packages/

Using cpu device
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Training the Autonomous Agent...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------
| time/              |      |
|    fps             | 63   |
|    iterations      | 1    |
|    time_elapsed    | 32   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 40          |
|    iterations           | 2           |
|    time_elapsed         | 102         |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009998031 |
|    clip_fraction        | 0.141       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.6        |
|    explained_variance   | -2.07e-05   |
|    learning_rate        | 0.0003      |
|    loss                 | 0.122       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00964    |
|    value_loss           | 0.524       |
-----------------------------------------
----------------------------------

In [3]:
# Evaluation and Video Recording
def show_video(video_path):
    video_file = open(video_path, "r+b").read()
    encoded = base64.b64encode(video_file).decode("ascii")
    ipythondisplay.display(HTML(data=f'''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{encoded}" type="video/mp4" />
             </video>'''))

# Record a test run
eval_env = DummyVecEnv([lambda: gym.make("CarRacing-v3", render_mode="rgb_array", continuous=False)])
eval_env = VecVideoRecorder(eval_env, "./video/", record_video_trigger=lambda x: x == 0, video_length=500)

obs = eval_env.reset()
for _ in range(500):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = eval_env.step(action)

eval_env.close()

# Show the AI driving!
video_path = str(list(Path("./video/").glob("*.mp4"))[0])
show_video(video_path)

Saving video to /content/video/rl-video-step-0-to-step-500.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"


Moviepy - Building video /content/video/rl-video-step-0-to-step-500.mp4.
Moviepy - Writing video /content/video/rl-video-step-0-to-step-500.mp4



/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Done !
Moviepy - video ready /content/video/rl-video-step-0-to-step-500.mp4
